# Modelo Preditivo - Tempo de Entrega (ETA)
Notebook para criar modelo preditivo que melhore a estimativa de tempo de entrega.

**Objetivo:** Melhorar a precisão da estimativa de tempo de entrega (`eta_minutes_quote`) apenas para pedidos dos canais próprios (site_proprio + whatsapp).

**Metodologia:** Divisão train/test dupla, cross-validation e comparação de 5 modelos usando RMSE.


## 1. Setup e Imports


In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

# Configurações visuais
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["axes.grid"] = True
plt.rcParams["font.size"] = 10

print("✅ Bibliotecas importadas com sucesso!")


✅ Bibliotecas importadas com sucesso!


## 2. Carregamento dos Dados


In [105]:
# Carregar dados limpos
df = pd.read_csv("../tratamento_inicial/Base_Kaiserhaus_Limpa.csv")

print("📊 Dados carregados:")
print(f"   • Total de registros: {len(df):,}")
print(f"   • Colunas: {df.shape[1]}")
print(f"   • Valores nulos: {df.isnull().sum().sum()}")

print("\n📋 Colunas disponíveis:")
print(list(df.columns))


📊 Dados carregados:
   • Total de registros: 5,000
   • Colunas: 16
   • Valores nulos: 0

📋 Colunas disponíveis:
['macro_bairro', 'nome_cliente', 'bairro_destino', 'order_datetime', 'platform', 'order_mode', 'distance_km', 'tempo_preparo_minutos', 'status', 'eta_minutes_quote', 'actual_delivery_minutes', 'total_brl', 'classe_pedido', 'platform_commission_pct', 'num_itens', 'satisfacao_nivel']


## 3. Filtro para Canais Próprios


In [106]:
# Filtrar apenas canais próprios
df_canais_proprios = df[df['platform'].isin(['site_proprio', 'whatsapp'])].copy()

print("🎯 FILTRO PARA CANAIS PRÓPRIOS")
print("=" * 50)
print(f"Total de pedidos canais próprios: {len(df_canais_proprios):,}")
print(f"Percentual do total: {len(df_canais_proprios)/len(df)*100:.1f}%")

print("\n📊 Distribuição por plataforma:")
distribuicao = df_canais_proprios['platform'].value_counts()
for plataforma, count in distribuicao.items():
    pct = count / len(df_canais_proprios) * 100
    print(f"   • {plataforma}: {count:,} pedidos ({pct:.1f}%)")

print("\n📈 Estatísticas do ETA nos canais próprios:")
print(f"   • Média: {df_canais_proprios['eta_minutes_quote'].mean():.2f} minutos")
print(f"   • Mediana: {df_canais_proprios['eta_minutes_quote'].median():.2f} minutos")
print(f"   • Desvio padrão: {df_canais_proprios['eta_minutes_quote'].std():.2f} minutos")
print(f"   • Range: {df_canais_proprios['eta_minutes_quote'].min()}-{df_canais_proprios['eta_minutes_quote'].max()} minutos")


🎯 FILTRO PARA CANAIS PRÓPRIOS
Total de pedidos canais próprios: 1,592
Percentual do total: 31.8%

📊 Distribuição por plataforma:
   • site_proprio: 1,052 pedidos (66.1%)
   • whatsapp: 540 pedidos (33.9%)

📈 Estatísticas do ETA nos canais próprios:
   • Média: 30.78 minutos
   • Mediana: 28.00 minutos
   • Desvio padrão: 9.21 minutos
   • Range: 12-57 minutos


## 4. Feature Engineering e One-Hot Encoding


In [107]:
# Função para obter período do dia
def obter_periodo_dia(hora):
    if 0 <= hora < 6:
        return 'Madrugada'
    elif 6 <= hora < 12:
        return 'Manhã'
    elif 12 <= hora < 18:
        return 'Tarde'
    else:
        return 'Noite'

# Criar período do dia diretamente no DataFrame
df_canais_proprios['periodo_dia'] = pd.to_datetime(df_canais_proprios['order_datetime']).dt.hour.apply(obter_periodo_dia)

# Selecionar features
features_numericas = ['distance_km', 'tempo_preparo_minutos', 'total_brl', 'num_itens']
features_categoricas = ['bairro_destino', 'periodo_dia']
target = 'eta_minutes_quote'

# Criar DataFrames separados
X_numericas = df_canais_proprios[features_numericas].copy()
X_categoricas = df_canais_proprios[features_categoricas].copy()
y = df_canais_proprios[target].copy()

# Aplicar One-Hot Encoding nas colunas categóricas SEM remover nenhuma categoria
encoder = OneHotEncoder(sparse_output=False, drop=None)
X_categoricas_encoded = encoder.fit_transform(X_categoricas)

# Criar DataFrame com nomes das colunas
feature_names = encoder.get_feature_names_out(features_categoricas)
X_categoricas_df = pd.DataFrame(X_categoricas_encoded, columns=feature_names, index=X_categoricas.index)

# Combinar features numéricas e categóricas
X_final = pd.concat([X_numericas, X_categoricas_df], axis=1)

# Salvar base com One-Hot Encoding aplicado
df_base_final = pd.concat([df_canais_proprios[features_numericas + ['eta_minutes_quote']], X_categoricas_df], axis=1)
df_base_final.to_csv("base_modelo_preditivo_eta.csv", index=False)

print("🔧 FEATURE ENGINEERING COMPLETO")
print("=" * 50)
print("✅ Arquivo: 'base_modelo_preditivo_eta.csv'")
print(f"📊 Total de registros: {len(df_base_final):,}")
print(f"📊 Colunas: {df_base_final.shape[1]}")
print(f"📊 Features categóricas criadas: {len(feature_names)}")

print("\n🕐 Distribuição por período do dia:")
distribuicao_periodo = df_canais_proprios['periodo_dia'].value_counts()
for periodo, count in distribuicao_periodo.items():
    pct = count / len(df_canais_proprios) * 100
    print(f"   • {periodo}: {count:,} pedidos ({pct:.1f}%)")

print(f"\n📊 Features originais: {len(features_numericas) + len(features_categoricas)}")
print(f"📊 Features após encoding: {X_final.shape[1]}")
print(f"📊 Features categóricas criadas: {len(feature_names)}")

print(f"\n📋 Features finais:")
print("   • Numéricas:", features_numericas)
print(f"   • Categóricas (encoded): {len(feature_names)} colunas")

print("\n🎯 Esta base contém:")
print("   • Pedidos do site próprio + WhatsApp")
print("   • Dados limpos e tratados")
print("   • TUDO NUMÉRICO (One-Hot Encoding aplicado)")
print("   • TODAS as categorias incluídas (incluindo Madrugada)")
print("   • X_final e y criados para modelos")
print("   • Prontos para modelos de ML")

🔧 FEATURE ENGINEERING COMPLETO
✅ Arquivo: 'base_modelo_preditivo_eta.csv'
📊 Total de registros: 1,592
📊 Colunas: 29
📊 Features categóricas criadas: 24

🕐 Distribuição por período do dia:
   • Tarde: 593 pedidos (37.2%)
   • Noite: 503 pedidos (31.6%)
   • Manhã: 448 pedidos (28.1%)
   • Madrugada: 48 pedidos (3.0%)

📊 Features originais: 6
📊 Features após encoding: 28
📊 Features categóricas criadas: 24

📋 Features finais:
   • Numéricas: ['distance_km', 'tempo_preparo_minutos', 'total_brl', 'num_itens']
   • Categóricas (encoded): 24 colunas

🎯 Esta base contém:
   • Pedidos do site próprio + WhatsApp
   • Dados limpos e tratados
   • TUDO NUMÉRICO (One-Hot Encoding aplicado)
   • TODAS as categorias incluídas (incluindo Madrugada)
   • X_final e y criados para modelos
   • Prontos para modelos de ML


## 5. Divisão Train/Test Dupla e Modelos


In [108]:
# Divisão Train/Test Dupla
# Primeira divisão: 80% treino / 20% teste
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(
    X_final, y, test_size=0.2, random_state=42
)

# Segunda divisão: 70% treino / 30% teste
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
    X_final, y, test_size=0.3, random_state=42
)

print("⚙️ DIVISÃO TRAIN/TEST DUPLA")
print("=" * 50)

print("\n📊 Primeira divisão (80/20):")
print(f"   • Treino: {len(X_train_1):,} registros ({len(X_train_1)/len(X_final)*100:.1f}%)")
print(f"   • Teste: {len(X_test_1):,} registros ({len(X_test_1)/len(X_final)*100:.1f}%)")

print("\n📊 Segunda divisão (70/30):")
print(f"   • Treino: {len(X_train_2):,} registros ({len(X_train_2)/len(X_final)*100:.1f}%)")
print(f"   • Teste: {len(X_test_2):,} registros ({len(X_test_2)/len(X_final)*100:.1f}%)")

# Definir modelos
# Modelos com variações e pipelines
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

modelos = {
    # Dummy Regressor - variações
    'Dummy Regressor (Mean)': DummyRegressor(strategy='mean'),
    
    # Linear Regression - variações
    'Linear Regression': LinearRegression(),
    'Linear Regression (Pipeline)': Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ]),

    # Linear Regression com Polynomial Features
    'Linear Regression (Polynomial2)': Pipeline([
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ]),

    'Linear Regression (Polynomial3)': Pipeline([
        ('poly', PolynomialFeatures(degree=3, include_bias=False)),
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())
    ]),
    
    # Decision Tree - variações
    'Decision Tree (Default)': DecisionTreeRegressor(random_state=42),
    'Decision Tree (Pruned)': DecisionTreeRegressor(
        max_depth=10, 
        min_samples_split=20, 
        min_samples_leaf=10, 
        random_state=42
    ),
    'Decision Tree (Deep)': DecisionTreeRegressor(
        max_depth=15, 
        min_samples_split=5, 
        min_samples_leaf=2, 
        random_state=42
    ),
    
    # Random Forest - variações
    'Random Forest (100)': RandomForestRegressor(n_estimators=100, random_state=42),
    'Random Forest (200)': RandomForestRegressor(n_estimators=200, random_state=42),
    'Random Forest (500)': RandomForestRegressor(n_estimators=500, random_state=42),
    'Random Forest (Tuned)': RandomForestRegressor(
        n_estimators=200,
        max_depth=20,
        min_samples_split=10,
        min_samples_leaf=5,
        random_state=42
    ),
    'Random Forest (Conservative)': RandomForestRegressor(
    n_estimators=200,
    max_depth=6,  # ← Reduzir de 15 para 6
    min_samples_split=20,  # ← Aumentar de 10 para 20
    min_samples_leaf=10,   # ← Aumentar de 5 para 10
    max_features='sqrt',   # ← Limitar features
    random_state=42
    ),
    
    
}

print(f"\n🤖 MODELOS DEFINIDOS:")
for nome, modelo in modelos.items():
    print(f"   • {nome}: {type(modelo).__name__}")

print(f"\n📊 Total de modelos: {len(modelos)}")
print(f"📊 Total de features: {X_final.shape[1]}")


⚙️ DIVISÃO TRAIN/TEST DUPLA

📊 Primeira divisão (80/20):
   • Treino: 1,273 registros (80.0%)
   • Teste: 319 registros (20.0%)

📊 Segunda divisão (70/30):
   • Treino: 1,114 registros (70.0%)
   • Teste: 478 registros (30.0%)

🤖 MODELOS DEFINIDOS:
   • Dummy Regressor (Mean): DummyRegressor
   • Linear Regression: LinearRegression
   • Linear Regression (Pipeline): Pipeline
   • Linear Regression (Polynomial2): Pipeline
   • Linear Regression (Polynomial3): Pipeline
   • Decision Tree (Default): DecisionTreeRegressor
   • Decision Tree (Pruned): DecisionTreeRegressor
   • Decision Tree (Deep): DecisionTreeRegressor
   • Random Forest (100): RandomForestRegressor
   • Random Forest (200): RandomForestRegressor
   • Random Forest (500): RandomForestRegressor
   • Random Forest (Tuned): RandomForestRegressor
   • Random Forest (Conservative): RandomForestRegressor

📊 Total de modelos: 13
📊 Total de features: 28


## 6. Treinamento, Validação e Comparação


In [109]:
# Dicionário para armazenar resultados da divisão 2
resultados_div2 = {}

print("📈 TREINAMENTO E VALIDAÇÃO")
print("=" * 70)

# Treinar todos os modelos na divisão 2
print("\n🔄 TREINAMENTO - DIVISÃO 2 (70/30):")
for nome, modelo in modelos.items():
    print(f"\n   Treinando {nome}...")
    
    # Treinar modelo na divisão 2
    modelo.fit(X_train_2, y_train_2)
    
    # Predições no conjunto de TREINO
    y_pred_train = modelo.predict(X_train_2)
    rmse_train = np.sqrt(mean_squared_error(y_train_2, y_pred_train))
    
    # Predições no conjunto de TESTE
    y_pred_test = modelo.predict(X_test_2)
    rmse_test = np.sqrt(mean_squared_error(y_test_2, y_pred_test))
    
    # Armazenar resultados
    resultados_div2[nome] = {
        'RMSE_Treino': rmse_train,
        'RMSE_Teste': rmse_test
    }
    
    print(f"      ✅ RMSE Treino: {rmse_train:.3f} | RMSE Teste: {rmse_test:.3f}")

print("\n📊 RESULTADOS DOS MODELOS:")
print("=" * 50)
for nome, resultados in resultados_div2.items():
    print(f"   • {nome}: Treino = {resultados['RMSE_Treino']:.3f} | Teste = {resultados['RMSE_Teste']:.3f}")

print(f"\n🎯 Escolha o melhor modelo para testar na divisão 1!")
print(f"📋 Modelos disponíveis: {list(resultados_div2.keys())}")

📈 TREINAMENTO E VALIDAÇÃO

🔄 TREINAMENTO - DIVISÃO 2 (70/30):

   Treinando Dummy Regressor (Mean)...
      ✅ RMSE Treino: 9.060 | RMSE Teste: 9.546

   Treinando Linear Regression...
      ✅ RMSE Treino: 4.498 | RMSE Teste: 4.432

   Treinando Linear Regression (Pipeline)...
      ✅ RMSE Treino: 4.498 | RMSE Teste: 4.432

   Treinando Linear Regression (Polynomial2)...
      ✅ RMSE Treino: 3.916 | RMSE Teste: 4.798

   Treinando Linear Regression (Polynomial3)...
      ✅ RMSE Treino: 2.819 | RMSE Teste: 35.381

   Treinando Decision Tree (Default)...
      ✅ RMSE Treino: 0.000 | RMSE Teste: 5.467

   Treinando Decision Tree (Pruned)...
      ✅ RMSE Treino: 3.721 | RMSE Teste: 4.414

   Treinando Decision Tree (Deep)...
      ✅ RMSE Treino: 2.091 | RMSE Teste: 5.492

   Treinando Random Forest (100)...
      ✅ RMSE Treino: 1.669 | RMSE Teste: 4.342

   Treinando Random Forest (200)...
      ✅ RMSE Treino: 1.639 | RMSE Teste: 4.358

   Treinando Random Forest (500)...
      ✅ RMSE Trein

## 7. Escolha e Teste do Melhor Modelo


In [110]:
# ESCOLHA O MODELO QUE VOCÊ QUER TESTAR
# Substitua 'Random Forest' pelo nome do modelo que você escolheu
modelo_escolhido = 'Random Forest (Tuned)'  # ← ALTERE AQUI

# Verificar se o modelo existe
if modelo_escolhido not in resultados_div2:
    print(f"❌ Erro: Modelo '{modelo_escolhido}' não encontrado!")
    print(f"📋 Modelos disponíveis: {list(resultados_div2.keys())}")
else:
    print(f"🎯 TESTANDO MODELO ESCOLHIDO: {modelo_escolhido}")
    print("=" * 50)
    
    # RMSE na divisão 2 (já calculado)
    rmse_div2_treino = resultados_div2[modelo_escolhido]['RMSE_Treino']
    rmse_div2_teste = resultados_div2[modelo_escolhido]['RMSE_Teste']
    print(f"📊 RMSE na divisão 2 (70/30): Treino = {rmse_div2_treino:.3f} | Teste = {rmse_div2_teste:.3f}")
    
    # Testar na divisão 1
    print(f"\n🔄 TESTANDO NA DIVISÃO 1 (80/20):")
    modelo_final = modelos[modelo_escolhido]
    modelo_final.fit(X_train_1, y_train_1)
    
    # RMSE de treino na divisão 1
    y_pred_train_1 = modelo_final.predict(X_train_1)
    rmse_div1_treino = np.sqrt(mean_squared_error(y_train_1, y_pred_train_1))
    
    # RMSE de teste na divisão 1
    y_pred_final = modelo_final.predict(X_test_1)
    rmse_div1_teste = np.sqrt(mean_squared_error(y_test_1, y_pred_final))
    
    print(f"📊 RMSE na divisão 1 (80/20): Treino = {rmse_div1_treino:.3f} | Teste = {rmse_div1_teste:.3f}")
    
    # Calcular RMSE médio (usando apenas os testes)
    rmse_medio = (rmse_div1_teste + rmse_div2_teste) / 2
    print(f"📊 RMSE médio (testes): {rmse_medio:.3f}")
    
    # Baseline (desvio padrão atual)
    baseline_rmse = df_canais_proprios['eta_minutes_quote'].std()
    melhoria_percentual = ((baseline_rmse - rmse_medio) / baseline_rmse) * 100
    
    print(f"\n📈 MELHORIA EM RELAÇÃO AO BASELINE:")
    print(f"   • Baseline (desvio padrão): {baseline_rmse:.3f} minutos")
    print(f"   • Modelo escolhido: {rmse_medio:.3f} minutos")
    print(f"   • Melhoria: {melhoria_percentual:.1f}%")
    
    print(f"\n💡 INSIGHTS PARA O NEGÓCIO:")
    print(f"   • O modelo {modelo_escolhido} foi testado")
    print(f"   • Redução de {melhoria_percentual:.1f}% no erro de estimativa")
    print(f"   • Aplicável apenas para canais próprios (site + WhatsApp)")
    print(f"   • Features mais importantes: distância e tempo de preparo")

🎯 TESTANDO MODELO ESCOLHIDO: Random Forest (Tuned)
📊 RMSE na divisão 2 (70/30): Treino = 3.254 | Teste = 4.278

🔄 TESTANDO NA DIVISÃO 1 (80/20):
📊 RMSE na divisão 1 (80/20): Treino = 3.241 | Teste = 4.324
📊 RMSE médio (testes): 4.301

📈 MELHORIA EM RELAÇÃO AO BASELINE:
   • Baseline (desvio padrão): 9.211 minutos
   • Modelo escolhido: 4.301 minutos
   • Melhoria: 53.3%

💡 INSIGHTS PARA O NEGÓCIO:
   • O modelo Random Forest (Tuned) foi testado
   • Redução de 53.3% no erro de estimativa
   • Aplicável apenas para canais próprios (site + WhatsApp)
   • Features mais importantes: distância e tempo de preparo
